<a href="https://colab.research.google.com/github/HAL22/Kaggle-Competitions/blob/contradictory/Contradictory_My_Dear_Watson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install kaggle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
!pip install datasets transformers[sentencepiece]
!apt install git-lfs
from transformers import AutoTokenizer
import pandas as pd
from datasets import load_dataset
from datasets import Dataset, DatasetDict, load_dataset
from sklearn.model_selection import train_test_split
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
!git config --global user.email "thethelafaltein@gmail.com"
!git config --global user.name "HAL22"

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
%%capture
!unzip '/content/test.csv.zip'
!unzip '/content/train.csv.zip'

In [6]:
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")

In [7]:
df_train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [8]:
df_train = df_train[["premise", "hypothesis","label"]].copy()

In [9]:
df_train.head()

,premise,hypothesis,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,1


In [10]:
train = Dataset.from_pandas(df_train)
train

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 12120
})

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")


In [12]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased",num_labels=3)

In [13]:
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["premise"],data["hypothesis"])

In [14]:
# For GPU
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", 
    hidden_dropout_prob=0.35,
    attention_probs_dropout_prob=0.35,
    num_labels=3,
).to(device)

In [17]:
tokenized_ds = train.map(tokenize_dataset, batched=True)

  0%|          | 0/13 [00:00<?, ?ba/s]

In [18]:
# Transformers assumes that labels have the column name "labels"
tokenized_ds = tokenized_ds.rename_columns({"label": "labels"})

In [19]:
dsd = tokenized_ds.train_test_split(.25, seed=42)
dsd

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9090
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3030
    })
})

In [20]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred):
    labels=pred.label_ids
    preds=pred.predictions.argmax(-1)
    f1=f1_score(labels, preds, average='weighted')
    ac=accuracy_score(labels, preds)
    return {"accuracy":ac, "f1":f1}

In [21]:
from transformers import TrainingArguments, Trainer

bs = 16
epochs = 1
lr = 4e-5

In [22]:
args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=6,
    weight_decay=0.01,)

trainer = Trainer(
    model=model, 
    args=args, 
    train_dataset=dsd['train'], 
    eval_dataset=dsd['test'],
    tokenizer=tokenizer, 
    compute_metrics=compute_metrics)

In [23]:

torch.cuda.empty_cache()

In [ ]:
trainer.train();

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.160500
1000,1.141900
1500,1.141000
2000,1.123100
2500,1.145100
3000,1.143200
3500,1.137500
4000,1.133800
4500,1.123800
5000,1.130800


Step,Training Loss
500,1.160500
1000,1.141900
1500,1.141000
2000,1.123100
2500,1.145100
3000,1.143200
3500,1.137500
4000,1.133800
4500,1.123800
5000,1.130800
